## Autores: 
* Mauro Barquinero
* Yandri Uchuari
* Marck Murillo
* Matias Tripode

# Trabajo Práctico Final: Linear/Quadratic Discriminant Analysis (LDA/QDA)

### Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

### Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

### LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

### Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

### Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

## Estructura del código

## Modelo

In [ ]:
import numpy as np
from numpy.linalg import det, inv

In [ ]:
class ClassEncoder:
  def fit(self, y):
    self.names = np.unique(y)
    self.name_to_class = {name:idx for idx, name in enumerate(self.names)}
    self.fmt = y.dtype
    # Q1: por que no hace falta definir un class_to_name para el mapeo inverso?

  def _map_reshape(self, f, arr):
    return np.array([f(elem) for elem in arr.flatten()]).reshape(arr.shape)
    # Q2: por que hace falta un reshape?

  def transform(self, y):
    return self._map_reshape(lambda name: self.name_to_class[name], y)

  def fit_transform(self, y):
    self.fit(y)
    return self.transform(y)

  def detransform(self, y_hat):
    return self._map_reshape(lambda idx: self.names[idx], y_hat)

In [ ]:
class BaseBayesianClassifier:
  def __init__(self):
    self.encoder = ClassEncoder()

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    # Q3: para que sirve bincount?
    return np.log(a_priori)

  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # first encode the classes
    y = self.encoder.fit_transform(y)

    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # check that a_priori has the correct number of classes
    assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=self.encoder.fmt)

    for i in range(m_obs):
      encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
      y_hat[i] = self.encoder.names[encoded_y_hat_i]

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [ ]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    # Q6: por que se usa bias=True en vez del default bias=False?
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    # Q7: que hace axis=1? por que no axis=0?

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [ ]:
class TensorizedQDA(QDA):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X,y)

        # stack onto new dimension
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x

        return 0.5*np.log(det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()

    def _predict_one(self, x):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

## Código para pruebas

Seteamos los datos

In [ ]:
# hiperparámetros
rng_seed = 6543

In [ ]:
from sklearn.datasets import load_iris, fetch_openml

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

def get_penguins():
    # get data
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True, parser='auto')

    # drop non-numeric columns
    df.drop(columns=["island","sex"], inplace=True)

    # drop rows with missing values
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]

    return df.values, tgt.to_numpy().reshape(-1,1)

# showing for iris
X_full, y_full = get_iris_dataset()

print(f"X: {X_full.shape}, Y:{y_full.shape}")

X: (150, 4), Y:(150, 1)


In [ ]:
# peek data matrix
X_full[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [ ]:
# peek target vector
y_full[:5]

array([['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa']], dtype='<U10')

Separamos el dataset en train y test para medir performance

In [ ]:
# preparing data, train - test validation
# 70-30 split
from sklearn.model_selection import train_test_split

def split_transpose(X, y, test_sz, random_state):
    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=random_state)

    # transpose so observations are column vectors
    return X_train.T, y_train.T, X_test.T, y_test.T

def accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.4, rng_seed)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(4, 90) (1, 90) (4, 60) (1, 60)


Entrenamos un QDA y medimos su accuracy

In [ ]:
qda = QDA()

qda.fit(train_x, train_y)

In [ ]:
train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0167


Con el magic %%timeit podemos estimar el tiempo que tarda en correr una celda en base a varias ejecuciones. Por poner un ejemplo, acá vamos a estimar lo que tarda un ciclo completo de QDA y también su inferencia (predicción).

Ojo! a veces [puede ser necesario ejecutarlo varias veces](https://stackoverflow.com/questions/10994405/python-timeit-results-cached-instead-of-calculated) para obtener resultados consistentes.

Si quieren explorar otros métodos de medición también es válido!

In [ ]:
%%timeit

qda.predict(test_x)

1.27 ms ± 55.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit

model = QDA()
model.fit(train_x, train_y)
model.predict(test_x)

1.33 ms ± 14.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Consigna


## Implementación base
1. Entrenar un modelo QDA sobre el dataset *iris* utilizando las distribuciones *a priori* a continuación ¿Se observan diferencias?¿Por qué cree? _Pista: comparar con las distribuciones del dataset completo, **sin splitear**_.
    1. Uniforme (cada clase tiene probabilidad 1/3)
    2. Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)
2. Repetir el punto anterior para el dataset *penguin*.
3. Implementar el modelo LDA, entrenarlo y testearlo contra los mismos sets que QDA (no múltiples prioris) ¿Se observan diferencias? ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?
4. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Las conclusiones previas se mantienen?
5. Estimar y comparar los tiempos de predicción de las clases `QDA` y `TensorizedQDA`. De haber diferencias ¿Cuáles pueden ser las causas?


**Sugerencia:** puede resultar de utilidad para cada inciso de comparación utilizar tablas del siguiente estilo:

<center>

Modelo | Dataset | Seed | Error (train) | Error (test)
:---: | :---: | :---: | :---: | :---:
QDA | Iris | 125 | 0.55 | 0.85
LDA | Iris | 125 | 0.22 | 0.8

</center>


## Optimización matemática

**Sugerencia:** considerar combinaciones adecuadas de `transpose`, `reshape` y, ocasionalmente, `flatten`. Explorar la dimensionalidad de cada elemento antes de implementar las clases.

### QDA

Debido a la forma cuadrática de QDA, no se puede predecir para *n* observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de *n x n* en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

1. Implementar el modelo `FasterQDA` (se recomienda heredarlo de TensorizedQDA) de manera de eliminar el ciclo for en el método predict.
2. Comparar los tiempos de predicción de `FasterQDA` con `TensorizedQDA` y `QDA`.
3. Mostrar (puede ser con un print) dónde aparece la mencionada matriz de *n x n*, donde *n* es la cantidad de observaciones a predecir.
4. Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de *n x n* usando matrices de *n x p*.
5.Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente. ¿Hay cambios en los tiempos de predicción?


### LDA

1. "Tensorizar" el modelo LDA y comparar sus tiempos de predicción con el modelo antes implementado. *Notar que, en modo tensorizado, se puede directamente precomputar $\mu^T \cdot \Sigma^{-1} \in \mathbb{R}^{k \times 1 \times p}$ y guardar eso en vez de $\Sigma^{-1}$.*
2. LDA no sufre del problema antes descrito de QDA debido a que no computa productos internos, por lo que no tiene un verdadero costo extra en memoria predecir "en batch". Implementar el modelo `FasterLDA` y comparar sus tiempos de predicción con las versiones anteriores de LDA.

## Preguntas teóricas

1. En LDA se menciona que la función a maximizar puede ser, mediante operaciones, convertida en:
$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$
Mostrar los pasos por los cuales se llega a dicha expresión.
2. Explicar, utilizando las respectivas funciones a maximizar, por qué QDA y LDA son "quadratic" y "linear".
3. La implementación de QDA estima la probabilidad condicional utilizando `0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x` que no es *exactamente* lo descrito en el apartado teórico ¿Cuáles son las diferencias y por qué son expresiones equivalentes?

El espíritu de esta componente práctica es la de establecer un mínimo de trabajo aceptable para su entrega; se invita al alumno a explorar otros aspectos que generen curiosidad, sin sentirse de ninguna manera limitado por la consigna.

## Ejercicio teórico

Sea una red neuronal de dos capas, la primera de 3 neuronas y la segunda de 1 con los parámetros inicializados con los siguientes valores:
$$
w^{(1)} =
\begin{pmatrix}
0.1 & -0.5 \\
-0.3 & -0.9 \\
0.8 & 0.02
\end{pmatrix},
b^{(1)} = \begin{pmatrix}
0.1 \\
0.5 \\
0.8
\end{pmatrix},
w^{(2)} =
\begin{pmatrix}
-0.4 & 0.2 & -0.5
\end{pmatrix},
b^{(2)} = 0.7
$$

y donde cada capa calcula su salida vía

$$
y^{(i)} = \sigma (w^{(i)} \cdot x^{(i)}+b^{(i)})
$$

donde $\sigma (z) = \frac{1}{1+e^{-z}}$ es la función sigmoidea .

\\
Dada la observación $x=\begin{pmatrix}
1.8 \\
-3.4
\end{pmatrix}$, $y=5$ y la función de costo $J(\theta)=\frac{1}{2}(\hat{y}_\theta-y)^2$, calcular las derivadas de J respecto de cada parámetro $w^{(1)}$, $w^{(2)}$, $b^{(1)}$, $b^{(2)}$.

*Nota: Con una sigmoidea a la salida jamás va a poder estimar el 5 "pedido", pero eso no afecta al mecanismo de backpropagation!*

## Preguntas en el código
Previamente las preguntas "técnicas" en comentarios en el código eran parte del TP, y buscaban que el alumno logre entrar en el detalle de por qué cada linea de código es como es y en el orden en el que está. Ya no forman parte de la consigna, pero se aconseja al alumno intentar responderlas. Las respuestas a las mismas se encuentran en un archivo separado.

-------------------------------

************ **Aca arrancan las respuestas y las implementaciones del TP**  ************

------------------------------------

## Implementacion Base

**CODIGO DE SETUP**

El siguiente codigo de setup se utiliza para **todas las pruebas** (tanto QDA como LDA).
Simplemente se descomentan las lineas para el dataset que queremos utilizar
ejemplo: Si queremos utilizar el penguin quedaria
```
    # Cargar el dataset iris
    # X_full, y_full = get_iris_dataset()
    # Cargar el dataset penguin
    X_full, y_full = get_penguins()
```

Si queremos probar sin split quedaria:
```
    # [Split]: Dividir el dataset entre test y train 
    # train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, test_sz, seed)
    # [Sin Split]: Dataset completo sin split
    train_x, train_y, test_x, test_y = X_full.T, y_full.T, X_full.T, y_full.T
```


In [ ]:
# *** CODIGO DE SETUP *** 

# Cargar el dataset iris
# X_full, y_full = get_iris_dataset()
# Cargar el dataset penguin
X_full, y_full = get_penguins()

# Seed
seed = 16
# Test size
test_sz = 0.3 # 30%
# [Split]: Dividir el dataset entre test y train 
train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, test_sz, seed)
# [Sin Split]: Dataset completo sin split
# train_x, train_y, test_x, test_y = X_full.T, y_full.T, X_full.T, y_full.T
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)


(4, 205) (1, 205) (4, 137) (1, 137)


**QDA + Uniforme (cada clase tiene probabilidad 1/3)**

In [ ]:
qda = QDA()
qda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])

y_train_pred = qda.predict(train_x)
y_test_pred = qda.predict(test_x)
qda_train_acc = accuracy(train_y, y_train_pred)
qda_test_acc = accuracy(test_y, y_test_pred)
print(f"* QDA: Train (apparent) error is {1-qda_train_acc:.4f} while test error is {1-qda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* QDA: Train (apparent) error is 0.0098 while test error is 0.0146
Error (train): 0.98%
Error (test): 1.46%


**QDA Primera clase con probabilidad 0.9, las demás 0.05**

In [ ]:
qda = QDA()
# Primera clase con probabilidad 0.9, las demás 0.05
qda.fit(train_x, train_y, a_priori=[0.9, 0.05,  0.05])

y_train_pred = qda.predict(train_x)
y_test_pred = qda.predict(test_x)
qda_train_acc = accuracy(train_y, y_train_pred)
qda_test_acc = accuracy(test_y, y_test_pred)
print(f"* QDA: Train (apparent) error is {1-qda_train_acc:.4f} while test error is {1-qda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")


* QDA: Train (apparent) error is 0.0146 while test error is 0.0292
Error (train): 1.46%
Error (test): 2.92%


**QDA Segunda clase con probabilidad 0.05, 0.9 y 0.05**

In [ ]:
qda = QDA()
# Segunda clase con probabilidad 0.05, 0.9 y 0.05
qda.fit(train_x, train_y, a_priori=[0.05, 0.9,  0.05])

y_train_pred = qda.predict(train_x)
y_test_pred = qda.predict(test_x)
qda_train_acc = accuracy(train_y, y_train_pred)
qda_test_acc = accuracy(test_y, y_test_pred)
print(f"* QDA: Train (apparent) error is {1-qda_train_acc:.4f} while test error is {1-qda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")


* QDA: Train (apparent) error is 0.0244 while test error is 0.0365
Error (train): 2.44%
Error (test): 3.65%


**QDA + Tercera clase con probabilidad 0.05, 0.05 y 0.9**

In [ ]:
qda = QDA()
# Tercera clase con probabilidad 0.05, 0.05 y 0.9
qda.fit(train_x, train_y, a_priori=[0.05, 0.05,  0.9])

y_train_pred = qda.predict(train_x)
y_test_pred = qda.predict(test_x)
qda_train_acc = accuracy(train_y, y_train_pred)
qda_test_acc = accuracy(test_y, y_test_pred)
print(f"* QDA: Train (apparent) error is {1-qda_train_acc:.4f} while test error is {1-qda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* QDA: Train (apparent) error is 0.0098 while test error is 0.0146
Error (train): 0.98%
Error (test): 1.46%


**Tabla comparativa de QDA**

Modelo | Dataset | Seed | a_priori | Split | Error (train) | Error (test)
:---: | :---: | :---: | :---: | :---: | :---: | :---:
QDA | Iris | 125 | [1/3, 1/3, 1/3] | 70/30 | 2.86% | 0.00%
QDA | Iris | 125 | [0.9, 0.05, 0.05] | 70/30 | 2.86% | 0.00%
QDA | Iris | 125 | [0.05, 0.9, 0.05] | 70/30 | 6.67% | 2.22%
QDA | Iris | 125 | [0.05, 0.05, 0.9] | 70/30 | 4.76% | 0.00%
QDA | Iris | 125 | [1/3, 1/3, 1/3] | 100 | 2.00% | 2.00%
QDA | Iris | 125 | [0.9, 0.05, 0.05] | 100 | 2.00% | 2.00%
QDA | Iris | 125 | [0.05, 0.9, 0.05] | 100 | 3.33% | 3.33%
QDA | Iris | 125 | [0.05, 0.05, 0.9] | 100 | 4.00% | 4.00%
 |  |  |  |  |  |
QDA | Penguin | 125 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 0.97%
QDA | Penguin | 125 | [0.9, 0.05, 0.05] | 70/30 | 1.67% | 1.94%
QDA | Penguin | 125 | [0.05, 0.9, 0.05] | 70/30 | 4.60% | 6.80%
QDA | Penguin | 125 | [0.05, 0.05, 0.9] | 70/30 | 0.84% | 0.97%
QDA | Penguin | 125 | [1/3, 1/3, 1/3] | 100 | 0.88% | 0.88%
QDA | Penguin | 125 | [0.9, 0.05, 0.05] | 100 | 1.75% | 1.75%
QDA | Penguin | 125 | [0.05, 0.9, 0.05] | 100 | 3.51% | 3.51%
QDA | Penguin | 125 | [0.05, 0.05, 0.9] | 100 | 0.88% | 0.88%




En la tabla de arriba vemos la comparativa de entrenar un modelo QDA sobre el dataset *iris* y  *penguin* utilizando distribuciones *a priori* (ver columna *a_priori*) y con splt 70-30 y sin split (indicado como 100).

**- ¿Se observan diferencias?**

Respuesta:
* Errores en función de la distribución a priori:
    - Las distribuciones a priori no uniformes ([0.9, 0.05, 0.05], [0.05, 0.9, 0.05], [0.05, 0.05, 0.9]) tienden a generar mayores errores en comparación con la distribución uniforme ([1/3, 1/3, 1/3]).
    - Los errores son más significativos en las configuraciones en las que se favorece una clase minoritaria, especialmente en el caso del iris con la configuración [0.05, 0.9, 0.05].
    
* Errores entre los splits 70/30 y 100:
    - Los errores de prueba en el split 70/30 suelen ser ligeramente mejores que en el caso del entrenamiento completo (100). Esto puede deberse a que el conjunto de prueba evalúa el modelo con datos nuevos, mientras que el 100 evalúa sobre el mismo conjunto, donde es más propenso a errores causados por datos desbalanceados.
    - La diferencia en errores es más marcada en el dataset iris que en penguin.

**- ¿Por qué cree?**

Respuesta:
- **Impacto de la distribución a priori**:
Cuando se usan distribuciones no uniformes, el modelo favorece una clase particular.

- **Diferencias entre split 70/30 y 100**: En el caso 70/30, el conjunto de prueba representa una validación más realista del modelo. Como los datos están separados, se reduce el sobreajuste que ocurre al evaluar en el mismo conjunto usado para entrenamiento (caso 100).
Sin embargo, en conjuntos más balanceados o grandes como penguin, la diferencia entre split 70/30 y 100 es menos significativa, ya que las distribuciones en el conjunto completo y en los subconjuntos son más parecidas.

- **Propiedades de los datasets**:
El dataset iris tiene menos muestras y podría ser más sensible al desbalance generado por distribuciones a priori no uniformes.
Penguin tiene más datos y mejor balance, lo que amortigua el impacto de las decisiones sobre las distribuciones a priori.


-------------------

**Ejercicio**

3. Implementar el modelo **LDA**

In [ ]:
class LDA(BaseBayesianClassifier):
    def _fit_params(self, X, y):
        y_flat = y.flatten()
        self.shared_cov = np.cov(X, bias=True)
        self.inv_shared_cov = np.linalg.inv(self.shared_cov)    

        self.means = [
            X[:, y_flat == idx].mean(axis=1, keepdims=True)
            for idx in range(len(self.log_a_priori))
        ]
    """
    Este método calcula el logaritmo de la probabilidad condicional para una observación x dada y una clase j.
    """
    def _predict_log_conditional(self, x, class_idx):
        unbiased_x = x - self.means[class_idx]
        return -0.5 * unbiased_x.T @ self.inv_shared_cov @ unbiased_x

**Ejercicio**

3. Comparar **LDA** contra los mismos sets que **QDA** 

**LDA + Uniforme (cada clase tiene probabilidad 1/3)**

In [ ]:
lda = LDA()
lda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])

y_train_pred = lda.predict(train_x)
y_test_pred = lda.predict(test_x)
lda_train_acc = accuracy(train_y, y_train_pred)
lda_test_acc = accuracy(test_y, y_test_pred)
print(f"* LDA: Train (apparent) error is {1-lda_train_acc:.4f} while test error is {1-lda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* LDA: Train (apparent) error is 0.0146 while test error is 0.0146
Error (train): 1.46%
Error (test): 1.46%


**LDA + Primera clase con probabilidad 0.9, las demás 0.05**

In [ ]:
lda = LDA()
# Primera clase con probabilidad 0.9, las demás 0.05
lda.fit(train_x, train_y, a_priori=[0.9, 0.05,  0.05])

y_train_pred = lda.predict(train_x)
y_test_pred = lda.predict(test_x)
lda_train_acc = accuracy(train_y, y_train_pred)
lda_test_acc = accuracy(test_y, y_test_pred)
print(f"* LDA: Train (apparent) error is {1-lda_train_acc:.4f} while test error is {1-lda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* LDA: Train (apparent) error is 0.4293 while test error is 0.4526
Error (train): 42.93%
Error (test): 45.26%


**LDA + Segunda clase con probabilidad 0.9, las demás 0.05**

In [ ]:
lda = LDA()
# Segunda clase con probabilidad 0.9, las demás 0.05
lda.fit(train_x, train_y, a_priori=[0.05, 0.9,  0.05])

y_train_pred = lda.predict(train_x)
y_test_pred = lda.predict(test_x)
lda_train_acc = accuracy(train_y, y_train_pred)
lda_test_acc = accuracy(test_y, y_test_pred)
print(f"* LDA: Train (apparent) error is {1-lda_train_acc:.4f} while test error is {1-lda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* LDA: Train (apparent) error is 0.4000 while test error is 0.3796
Error (train): 40.00%
Error (test): 37.96%


**LDA + Tercera clase con probabilidad 0.9, las demás 0.05**

In [ ]:
lda = LDA()
# Tercera clase con probabilidad 0.9, las demás 0.05
lda.fit(train_x, train_y, a_priori=[0.05, 0.05,  0.9])

y_train_pred = lda.predict(train_x)
y_test_pred = lda.predict(test_x)
lda_train_acc = accuracy(train_y, y_train_pred)
lda_test_acc = accuracy(test_y, y_test_pred)
print(f"* LDA: Train (apparent) error is {1-lda_train_acc:.4f} while test error is {1-lda_test_acc:.4f}")

# Calculate train and test errors
error_train = np.mean(y_train_pred != train_y)
error_test = np.mean(y_test_pred != test_y)

print(f"Error (train): {error_train:.2%}")
print(f"Error (test): {error_test:.2%}")

* LDA: Train (apparent) error is 0.4293 while test error is 0.4672
Error (train): 42.93%
Error (test): 46.72%


**Tabla Comparativa LDA**

Modelo | Dataset | Seed | a_priori | Split | Error (train) | Error (test)
:---: | :---: | :---: | :---: | :---: | :---: | :---:
LDA | Iris | 125 | [1/3, 1/3, 1/3] | 70/30 | 12.38% | 13.33%
LDA | Iris | 125 | [0.9, 0.05, 0.05] | 70/30 | 54.29% | 46.67%
LDA | Iris | 125 | [0.05, 0.9, 0.05] | 70/30 | 63.81% | 55.56%
LDA | Iris | 125 | [0.05, 0.05, 0.9] | 70/30 | 53.33% | 48.89%
LDA | Iris | 125 | [1/3, 1/3, 1/3] | 100 | 13.33% | 13.33%
LDA | Iris | 125 | [0.9, 0.05, 0.05] | 100 | 51.33% | 51.33%
LDA | Iris | 125 | [0.05, 0.9, 0.05] | 100 | 62.67% | 62.67%
LDA | Iris | 125 | [0.05, 0.05, 0.9] | 100 | 55.33% | 55.33%
 |  |  |  |  |  | 
LDA | Penguin | 125 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 0.97% 
LDA | Penguin | 125 | [0.9, 0.05, 0.05] | 70/30 | 44.77% | 37.86%
LDA | Penguin | 125 | [0.05, 0.9, 0.05] | 70/30 | 38.49% | 44.66%
LDA | Penguin | 125 | [0.05, 0.05, 0.9] | 70/30 | 45.19% | 39.81%
LDA | Penguin | 125 | [1/3, 1/3, 1/3] | 100 | 0.88% | 0.88%
LDA | Penguin | 125 | [0.9, 0.05, 0.05] | 100 | 42.98% | 42.98%
LDA | Penguin | 125 | [0.05, 0.9, 0.05] | 100 | 40.64% | 40.64%
LDA | Penguin | 125 | [0.05, 0.05, 0.9] | 100 | 43.57% | 43.57%

**¿Se observan diferencias con respecto a la tabla anterior de QDA?**

Respuesta:

Sí, hay diferencias claras entre los resultados de LDA y QDA:
- Errores significativamente mayores en LDA con distribuciones a priori no uniformes:
    - En el dataset iris, los errores de LDA aumentan considerablemente para distribuciones no uniformes como [0.9, 0.05, 0.05] o [0.05, 0.9, 0.05], superando el 50% en algunos casos.
    - En el dataset penguin, los errores de LDA también se incrementan notablemente con distribuciones no uniformes

- Errores con distribuciones uniformes [1/3, 1/3, 1/3]:
    - Para distribuciones uniformes, ambos modelos presentan errores bajos, aunque LDA tiene mayores errores que QDA en el dataset iris (13% vs 2%).

- Impacto del split (70/30 vs 100):
    - Para LDA, los errores entre el split 70/30 y 100 son similares. En QDA, el split 70/30 suele mostrar mejores resultados en el conjunto de prueba, especialmente en iris.

**¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?** 

Respuesta:

Sí, QDA es notoriamente mejor que LDA en este contexto, especialmente con distribuciones a priori no uniformes.
- QDA es más robusto frente a distribuciones a priori no uniformes y datasets desbalanceados.
- LDA puede ser más rápido y eficiente en casos donde las clases sean balanceadas y las distribuciones sean similares, pero es menos flexible que QDA.

-------------------------

**Ejercicio**

4. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Las conclusiones previas se mantienen?

Se realiza la comparacion con Seeds de 125, 64 y 16. En todos los casos el split es 70-30 t la distribucion a priori es 1/3 para todas las clases:

**Tabla Comparativa LDA y QDA**

Modelo | Dataset | Seed | a_priori | Split | Error (train) | Error (test)
:---: | :---: | :---: | :---: | :---: | :---: | :---:
QDA | Iris | 125 | [1/3, 1/3, 1/3] | 70/30 | 2.86% | 0.00%
QDA | Iris | 64 | [1/3, 1/3, 1/3] | 70/30 | 0.00% | 4.44%   
QDA | Iris | 16 |  [1/3, 1/3, 1/3] | 70/30 | 3.81% |2.22%   
QDA | Penguin | 125 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 0.97%
QDA | Penguin | 64 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 1.94%
QDA | Penguin | 16 |  [1/3, 1/3, 1/3] | 70/30 | 1.26% | 0.97%
|  |  |  |  |  | 
LDA | Iris | 125 | [1/3, 1/3, 1/3] | 70/30 | 12.38% | 13.33%
LDA | Iris | 64 | [1/3, 1/3, 1/3] | 70/30 | 14.29% | 11.11% 
LDA | Iris | 16 |  [1/3, 1/3, 1/3] | 70/30 | 10.48% | 20.00%
LDA | Penguin | 125 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 0.97%
LDA | Penguin | 64 | [1/3, 1/3, 1/3] | 70/30 | 0.84% | 0.97%
LDA | Penguin | 16 |  [1/3, 1/3, 1/3] | 70/30 | 1.26% | 0.00%


¿Se mantienen las conclusiones previas?
Las conclusiones se matizan al considerar los nuevos resultados con diferentes random seeds:

Respuesta:

- Errores en QDA:
    - `Dataset iris`:
    El error de entrenamiento y prueba varía con el random seed, pero en general, los errores son bajos.
    Con el seed 64, el error de prueba aumenta ligeramente (4.44%), lo que podría atribuirse a la naturaleza aleatoria del split y a una posible menor representatividad de las clases en el conjunto de prueba.
    - `Dataset penguin`:
    Los errores son consistentemente bajos, tanto en entrenamiento como en prueba, con pequeñas variaciones que no afectan la conclusión de que QDA es un modelo efectivo para este dataset.

- Errores en LDA:
    - `Dataset iris`:
    Los errores de entrenamiento y prueba también muestran variaciones según el random seed, con errores de prueba que van desde 11.11% hasta 20.00%.
    Aunque los errores de LDA son mayores que los de QDA, el rango de variación no es dramático y sigue reflejando su menor capacidad para modelar la complejidad del dataset en comparación con QDA.
    - `Dataset penguin`:
    Al igual que QDA, LDA tiene errores consistentemente bajos, con variaciones mínimas. De hecho, con el seed 16, LDA tiene un error de prueba del 0%, lo que sugiere que el modelo fue particularmente efectivo en ese split.

-------------------------

**Ejercicio**

5. Estimar y comparar los tiempos de predicción de las clases `QDA` y `TensorizedQDA`. De haber diferencias ¿Cuáles pueden ser las causas?

 Estimar los tiempos de predicción de `TensorizedQDA`

In [ ]:

tensorized_qda = TensorizedQDA()
tensorized_qda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])

In [ ]:
%%timeit
tensorized_qda.predict(train_x)

1.55 ms ± 6.78 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
qda = QDA()
qda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])

In [ ]:
%%timeit
qda.predict(train_x)

4.01 ms ± 24.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Modelo | Dataset | Seed | a_priori | Split | Performance 
:---: | :---: | :---: | :---: | :---: | :---: 
Tensorized QDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 2.7 ms ± 112 μs 
QDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 7.23 ms ± 146 μs 

**Ejercicio**

**De haber diferencias ¿Cuáles pueden ser las causas?**

Respuesta:
- El Tensorized QDA performa ~2.7 veces mas rapido que el QDA y esto se debe a que cuenta con una implementacion mas optimizada de la funcion `_predict_log_conditionals`
- El mejor desempeño de `TensorizedQDA` se debe principalmente a la vectorización y al uso eficiente de operaciones en batch, que minimizan la latencia y el overhead computacional, mientras que QDA realiza los cálculos de manera iterativa y menos eficiente

--------------

## Optimización matemática

---------------------------

**Ejercicio**
**Faster QDA**

1. Para implementar FasterQDA, heredando de TensorizedQDA, nuestro objetivo es eliminar el ciclo for del método predict. Esto se logrará aprovechando operaciones matriciales vectorizadas y ajustando adecuadamente las dimensiones de los tensores mediante métodos como reshape, transpose, o flatten.

In [ ]:
class FasterQDA(TensorizedQDA):
    def _predict_log_conditionals(self, X):
        """
        Calcula log(P(X|G)) para todos los datos y clases de manera vectorizada.
        """
        # Expandir X para alinearlo con tensor_means
        unbiased_X = X[np.newaxis, :, :] - self.tensor_means  # (n_classes, n_features, n_samples)

        # Reorganizar para aplicar la matriz de covarianza inversa
        unbiased_X = unbiased_X.transpose(0, 2, 1)  # (n_classes, n_samples, n_features)

        # Producto interno: x^T * inv_cov * x
        inner_prod = np.einsum(
            'ijk,ikl,ijl->ij', unbiased_X, self.tensor_inv_cov, unbiased_X
        )  # (n_classes, n_samples)

        # Calcular logaritmo de probabilidad condicional
        log_conditional = (
            0.5 * np.log(np.linalg.det(self.tensor_inv_cov))[:, np.newaxis] - 0.5 * inner_prod
        )  # (n_classes, n_samples)

        return log_conditional

    def predict(self, X):
        """
        Predice las clases de todas las muestras de manera vectorizada.
        """
        # Calcular log(P(G)) + log(P(X|G))
        log_posteriors = self.log_a_priori[:, np.newaxis] + self._predict_log_conditionals(X)

        # Retornar la clase con máxima probabilidad para cada muestra
        return np.argmax(log_posteriors, axis=0)


In [ ]:
faster_qda = FasterQDA()
faster_qda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])

In [ ]:
%%timeit
faster_qda.predict(train_x)

68.3 μs ± 2.9 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


------------------------

**Ejercicio**

2. Comparar los tiempos de predicción de `FasterQDA` con `TensorizedQDA` y `QDA`.
De la tabla comparativa de mas abajo vemos que el `FasterQDA` es el mas rapido de los tres.


Modelo | Dataset | Seed | a_priori | Split | Performance 
:---: | :---: | :---: | :---: | :---: | :---: 
Faster QDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 65.9 μs ± 154 ns
Tensorized QDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 2.7 ms ± 112 μs 
QDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 7.23 ms ± 146 μs 

--------------------

**Ejercicio**

3. Mostrar (puede ser con un print) dónde aparece la mencionada matriz de *n x n*, donde *n* es la cantidad de observaciones a predecir.

¿Dónde podría generarse una matriz 𝑛×𝑛? Si accidentalmente calculáramos todos los productos cruzados entre muestras (por ejemplo, usando @ con ejes incorrectos o expandiendo dimensiones de forma innecesaria), generaríamos una matriz 
𝑛×𝑛.

Podemos insertar un print en el lugar donde calculamos `inner_prod` y revisar las dimensiones de cada operación intermedia. Agreguemos el siguiente código:

In [ ]:
class FasterQDA(TensorizedQDA):
    def _predict_log_conditionals(self, X):
        """
        Calcula log(P(X|G)) para todos los datos y clases de manera vectorizada.
        """
        unbiased_X = X[np.newaxis, :, :] - self.tensor_means  # (n_classes, n_features, n_samples)
        unbiased_X = unbiased_X.transpose(0, 2, 1)  # (n_classes, n_samples, n_features)

        print("unbiased_X shape:", unbiased_X.shape)  # Dimensiones de unbiased_X
        print("tensor_inv_cov shape:", self.tensor_inv_cov.shape)  # Dimensiones de tensor_inv_cov

        # Calcular producto interno
        inner_prod = np.einsum(
            'ijk,ikl,ijl->ij', unbiased_X, self.tensor_inv_cov, unbiased_X
        )

        print("inner_prod shape:", inner_prod.shape)  # Debemos asegurarnos que sea (n_classes, n_samples)

        log_conditional = (
            0.5 * np.log(np.linalg.det(self.tensor_inv_cov))[:, np.newaxis] - 0.5 * inner_prod
        )

        return log_conditional


--------------------

**Ejercicio**

4. Demostrar que $diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)$ es decir, que se puede "esquivar" la matriz de *n x n* usando matrices de *n x p*.

!['Pregunta 1'](optimization_mat_1.jpg) 

!['Pregunta 1'](optimization_mat_2.jpg) 

-----------

**Ejercicio**

5. Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente. ¿Hay cambios en los tiempos de predicción?

**Implementación FasterQDA**

In [ ]:
class FasterQDA(TensorizedQDA):
    def _predict_log_conditionals(self, X):
        """
        Calcula log(P(X|G)) para todas las clases de manera eficiente usando matrices n x p.
        """
        # Expandir X para alinearlo con self.tensor_means
        unbiased_X = X[np.newaxis, :, :] - self.tensor_means  # (n_classes, n_features, n_samples)

        # Cálculo eficiente de x^T * inv_cov * x para cada muestra y clase
        inner_prod = np.sum(
            (unbiased_X.transpose(0, 2, 1) @ self.tensor_inv_cov) * unbiased_X.transpose(0, 2, 1),
            axis=2,
        )  # (n_classes, n_samples)

        # Calcular log-condicional
        log_conditional = (
            0.5 * np.log(np.linalg.det(self.tensor_inv_cov))[:, np.newaxis] - 0.5 * inner_prod
        )  # (n_classes, n_samples)

        return log_conditional

    def predict(self, X):
        """
        Predice las clases de todas las muestras de manera eficiente.
        """
        # Calcular log(P(G)) + log(P(X|G))
        log_posteriors = self.log_a_priori[:, np.newaxis] + self._predict_log_conditionals(X)

        # Retornar la clase con máxima probabilidad para cada muestra
        return np.argmax(log_posteriors, axis=0)


In [ ]:
optimized_faster_qda = FasterQDA()
optimized_faster_qda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])

In [ ]:
%%timeit
optimized_faster_qda.predict(train_x)

28.1 μs ± 195 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


--------------------

**Ejercicio**

2. Comparar los tiempos de predicción de `FasterQDA` con `TensorizedQDA` y `QDA`.

De la tabla comparativa de mas abajo vemos que el `Optimizado FasterQDA` es el mas rapido de los cuatros en el orden de los ~`28.2 μs` mientras que el mas lento `QDA` esta en los ~`7.23 ms`


Modelo | Dataset | Seed | a_priori | Split | Performance 
:---: | :---: | :---: | :---: | :---: | :---: 
Optimizado Faster QDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 28.2 μs ± 219 ns
Faster QDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 65.9 μs ± 154 ns
Tensorized QDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 2.7 ms ± 112 μs 
QDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 7.23 ms ± 146 μs 

--------------

### Tensorizar LDA 

1. "Tensorizar" el modelo LDA y comparar sus tiempos de predicción con el modelo antes implementado. *Notar que, en modo tensorizado, se puede directamente precomputar $\mu^T \cdot \Sigma^{-1} \in \mathbb{R}^{k \times 1 \times p}$ y guardar eso en vez de $\Sigma^{-1}$.*
2. LDA no sufre del problema antes descrito de QDA debido a que no computa productos internos, por lo que no tiene un verdadero costo extra en memoria predecir "en batch". Implementar el modelo `FasterLDA` y comparar sus tiempos de predicción con las versiones anteriores de LDA.

**TensorizedLDA**

In [ ]:
class TensorizedLDA(LDA):
    def _fit_params(self, X, y):
        super()._fit_params(X, y)  # Usa la lógica original de LDA para calcular shared_cov, inv_shared_cov y means.

        self.tensor_means = np.stack(self.means, axis=0)

        self.means_inv_cov = self.tensor_means.transpose(0, 2, 1) @ self.inv_shared_cov[np.newaxis, :, :]

    def _predict_log_conditionals(self, X):
        unbiased_X = X[np.newaxis, :, :] - self.tensor_means  # (k, p, n)
        quadratic_term = np.einsum('kpn,pm,kpn->kn', unbiased_X, self.inv_shared_cov, unbiased_X)

        log_conditionals = -0.5 * quadratic_term  # (k, n)
        return log_conditionals

    def predict(self, X):
        log_posteriors = self.log_a_priori[:, np.newaxis] + self._predict_log_conditionals(X)

        return np.argmax(log_posteriors, axis=0)


In [ ]:
tensorized_lda = TensorizedLDA()
tensorized_lda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])

In [ ]:
%%timeit
tensorized_lda.predict(train_x)

55.5 μs ± 15.6 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
lda = LDA()
lda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])

In [ ]:
%%timeit
lda.predict(train_x)

2.51 ms ± 313 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


De la tabla abajo vemos que claramente el Tensorized LDA performa muy superior al LDA unas 26 veces mas rapido (~  2.23 ms/86.2 μs).

Modelo | Dataset | Seed | a_priori | Split | Performance 
:---: | :---: | :---: | :---: | :---: | :---: 
Tensorized LDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 86.2 μs ± 2.98 μs 
LDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 2.23 ms ± 5.53

----------------------------

**FasterLDA**

2. LDA no sufre del problema antes descrito de QDA debido a que no computa productos internos, por lo que no tiene un verdadero costo extra en memoria predecir "en batch". Implementar el modelo `FasterLDA` y comparar sus tiempos de predicción con las versiones anteriores de LDA.


Para implementar el modelo FasterLDA, aprovecharemos el hecho de que LDA no tiene el problema del almacenamiento de grandes matrices intermedias, ya que no requiere calcular productos internos para cada observación y clase. Esto significa que podemos optimizar el cálculo de las probabilidades condicionales al precomputar operaciones clave.

Implementación de FasterLDA
El modelo `FasterLDA` precomputará eficientemente 
𝜇
𝑇
Σ^−1
y 
−
0.5
𝜇
𝑇
Σ^−1
𝜇
 μ durante el ajuste para evitar cálculos redundantes en predicciones.

Discusión:
La implementación de FasterLDA aprovecha la naturaleza lineal de las operaciones de LDA y es adecuada para escenarios de predicción "en batch". Aunque LDA ya es eficiente, esta optimización puede ser útil para grandes conjuntos de datos, reduciendo el tiempo computacional de predicción significativamente.

In [ ]:
class FasterLDA(LDA):
    def _fit_params(self, X, y):
        """
        Ajusta los parámetros del modelo LDA de forma optimizada.
        """
        super()._fit_params(X, y)  # Usar la implementación original para obtener inv_shared_cov y means

        # Precomputar mu^T @ Sigma^-1 (k, 1, p)
        self.means_inv_cov = np.array([mu.T @ self.inv_shared_cov for mu in self.means])  # (k, 1, p)

        # Precomputar -0.5 * mu^T @ Sigma^-1 @ mu (k,)
        self.quadratic_means = np.array([-0.5 * (mu.T @ self.inv_shared_cov @ mu).flatten() for mu in self.means])  # (k,)

    def _predict_log_conditionals(self, X):
        """
        Calcula log(P(X|G)) de manera optimizada.
        """
        # X tiene forma (p, n)
        linear_term = (self.means_inv_cov @ X).squeeze(axis=1)  # (k, n)
        log_conditionals = self.quadratic_means[:, np.newaxis] + linear_term  # (k, n)
        return log_conditionals

    def predict(self, X):
        """
        Predice las clases de las observaciones X usando el modelo optimizado.
        """
        log_posteriors = self.log_a_priori[:, np.newaxis] + self._predict_log_conditionals(X)
        return np.argmax(log_posteriors, axis=0)


In [ ]:
faster_lda = FasterLDA()
faster_lda.fit(train_x, train_y, a_priori=[0.33333333, 0.33333333,  0.33333333])

In [ ]:
%%timeit
faster_lda.predict(train_x)

12.8 μs ± 2.12 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


De la tabla abajo vemos que claramente el `Faster LDA` performa superior al `Tensorized LDA` unas 7 veces mas rapido (~  86.2 μs / 11.7 μs).

Modelo | Dataset | Seed | a_priori | Split | Performance 
:---: | :---: | :---: | :---: | :---: | :---: 
Faster LDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 11.7 μs ± 56.3 ns 
Tensorized LDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 86.2 μs ± 2.98 μs 
LDA | Penguins | 125 | [1/3, 1/3, 1/3] | 70/30 | 2.23 ms ± 5.53

--------------------

## Preguntas Teoricas

!['Pregunta 1'](pregunta_teorica_1.jpg) 

!['Pregunta 3'](pregunta_teorica_2.jpg) 

!['Pregunta 3'](pregunta_teorica_3.jpg) 

!['Pregunta 1'](ejercicio_teorico_1.jpg) 

!['Pregunta 1'](ejercicio_teorico_2.jpg) 

!['Pregunta 1'](ejercicio_teorico_3.jpg) 